# 3 Виконання

## 3.1 Генерування набору даних

### Згенеруємо набір даних результатів вступу учнів до вищих навчальних закладів.

### Статистика минулих років показує, що в середньому до Університету подають документи 1500 абітурієнтів.

### Для початку імпортуємо модуль NumPy. Кількість згенерованих значення визначенється за допомогою змінної max_applicants.

In [177]:
import numpy as np

np.set_printoptions(
    formatter={'float': lambda x: "{0:0.2f}".format(x)})

max_applicants = 2400

*Визначення загальної кількості абітурієнтів*

### Згенеруємо значення в проміжку від 100 до 200 за допомогою numpy.random.random_integers. Надамо набору даних назви стовпців.

In [178]:
import pandas as pd
df = pd.DataFrame(
    np.random.randint(100, 200 + 1, (max_applicants, 3)))
df.columns = ['math', 'eng', 'ua']
df

,math,eng,ua
0,191,181,102
1,161,103,158
2,177,194,130
3,126,115,116
4,185,160,134
...,...,...,...
2395,156,100,196
2396,161,145,102
2397,118,194,134
2398,115,107,130


*Список балів абітурієнтів*

### Додамо поле, що визначає рейтинг. Рейтинг абітурієнтів формується за формулою 0,4БМ + 0,3БА + 0,3БУ, де БМ-бал з іспиту з математики, БА-бал з іспиту з англійської мови, БУ-бал з іспиту з української мови.

In [179]:
coefs = np.array([0.4, 0.3, 0.3])
df['score'] = np.apply_along_axis(
    lambda row: np.dot(row, coefs), 1, df.values)
df

,math,eng,ua,score
0,191,181,102,161.3
1,161,103,158,142.7
2,177,194,130,168.0
3,126,115,116,119.7
4,185,160,134,162.2
...,...,...,...,...
2395,156,100,196,151.2
2396,161,145,102,138.5
2397,118,194,134,145.6
2398,115,107,130,117.1


*Додавання поля середнього балу до балів студента*

### Згенеруємо стовпчик про наявність пільг в абітурієнта.

### Очевидно, що студентів з пільгами не може бути багато, то згенеруємо випадкові значення у проміжку від 0 до 1. Нехай значення, що менші privilage_quant, визначають, чи абітурієнти мають пільги.

In [180]:
privilage_quant = 0.3
privilige = np.random.random_sample((max_applicants, 1))
privilige = np.apply_along_axis(
    lambda x: x < privilage_quant, 1, privilige)
df['priv'] = privilige
df

,math,eng,ua,score,priv
0,191,181,102,161.3,False
1,161,103,158,142.7,False
2,177,194,130,168.0,False
3,126,115,116,119.7,True
4,185,160,134,162.2,False
...,...,...,...,...,...
2395,156,100,196,151.2,False
2396,161,145,102,138.5,True
2397,118,194,134,145.6,False
2398,115,107,130,117.1,False


*Додавання поля пільг до балів студента*

### Утворимо список значень результатів прийняття абітурієнта до освітнього закладу.

### Мінімальний прохідний бал на вступ 160 для абітурієнтів без пільг.

### З математики для абітурієнтів без пільг мінімальний бал іспиту не може бути менший 140 балів.

### Абітурієнти, які мають пільги, зараховуються при мінімумі 120 балів з усіх іспитів і їх рейтинг не може бути меншим ніж 144 бали.

### Університет може прийняти на навчання 350 абітурієнтів, з них не більше 10% це абітурієнти з пільгами.

In [181]:
mins_no_privilige = np.array([140, 120, 120, 160])
mins_privilige = np.array([120, 120, 120, 144]) 
    
def is_score_valid(row: np.ndarray):
    mins = mins_no_privilige
    if row[4]:
        mins = mins_privilige
    return (row[:4] > mins).all()
df['accepted'] = np.apply_along_axis(
    is_score_valid, 1, df.values)
df

,math,eng,ua,score,priv,accepted
0,191,181,102,161.3,False,False
1,161,103,158,142.7,False,False
2,177,194,130,168.0,False,True
3,126,115,116,119.7,True,False
4,185,160,134,162.2,False,True
...,...,...,...,...,...,...
2395,156,100,196,151.2,False,False
2396,161,145,102,138.5,True,False
2397,118,194,134,145.6,False,False
2398,115,107,130,117.1,False,False


*Визначення прийняття за кількістю балів*

### Оскільки порядок подачі документів абітурієнтами має значення, то додамо стовпець index, де чим він менший, тим раніше абітурієнт подав документи. Зробимо це за допомогою методу reset_index.

### Утворимо відсортований набір даних за стовпцями в такому порядку priv, score, index. Та визначимо ліміт прийнятих абітурієнтів. Для яких місць не залишиться, тим виставиться False у полі accepted.

In [182]:
df.reset_index(inplace=True)
df.sort_values(['priv', 'score', 'index'], inplace=True,
        ascending=[False, False, True])
df.reset_index(inplace=True, drop=True)
df.columns = ['original_index', 'math', 'eng',
                'ua', 'score', 'priv', 'accepted']
df

,original_index,math,eng,ua,score,priv,accepted
0,904,200,199,195,198.2,True,True
1,26,193,194,190,192.4,True,True
2,770,193,195,186,191.5,True,True
3,1762,198,193,180,191.1,True,True
4,2202,192,192,182,189.0,True,True
...,...,...,...,...,...,...,...
2395,2044,102,116,107,107.7,False,False
2396,681,104,113,106,107.3,False,False
2397,1798,112,100,108,107.2,False,False
2398,2009,105,111,106,107.1,False,False


*Індексація та сортування абітурієнтів*

### Визначення студентів з пільгами, що пройшли відбір.

In [183]:
priv = df.loc[(df.priv == True) & (df.accepted == True)].copy()
priv.reset_index(inplace=True, drop=True)
priv = priv[:35]
priv

,original_index,math,eng,ua,score,priv,accepted
0,904,200,199,195,198.2,True,True
1,26,193,194,190,192.4,True,True
2,770,193,195,186,191.5,True,True
3,1762,198,193,180,191.1,True,True
4,2202,192,192,182,189.0,True,True
5,844,196,196,171,188.5,True,True
6,2046,185,181,198,187.7,True,True
7,1260,185,198,180,187.4,True,True
8,2195,195,177,185,186.6,True,True
9,2280,186,190,183,186.3,True,True


*Перші 35 студентів з пільгами, що пройшли конкурс*

### Визначення результатів студентів без пільг та з пільгами, що не пройшли конкурс.

In [184]:
no_priv = df.loc[~df.original_index.isin(priv.original_index.to_list())].copy()
no_priv.sort_values(['score', 'original_index'], inplace=True,
        ascending=[False, True])
no_priv_accepted = no_priv.loc[no_priv.accepted == True]
no_priv_accepted.reset_index(inplace=True, drop=True)
no_priv_accepted = no_priv_accepted[:315]
no_priv_accepted

,original_index,math,eng,ua,score,priv,accepted
0,1286,200,182,197,193.7,False,True
1,2129,195,186,199,193.5,False,True
2,975,199,191,188,193.3,False,True
3,589,187,196,199,193.3,False,True
4,295,194,196,189,193.1,False,True
...,...,...,...,...,...,...,...
310,2000,152,169,192,169.1,False,True
311,178,193,171,135,169.0,False,True
312,1808,190,122,188,169.0,True,True
313,236,186,176,139,168.9,True,True


*Результати вступу*

### Об'єднання прийнятих абітурієнтів та виставлення accepted = False, тим, що не пройшли у максимальну кількість студентів.

In [185]:
accepted = pd.concat([priv, no_priv_accepted])
not_accepted = df.loc[~df.original_index.isin(accepted.original_index)]
not_accepted.accepted = False
df = pd.concat([accepted, not_accepted])
df

/tmp/ipykernel_130136/4227975460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_accepted.accepted = False


,original_index,math,eng,ua,score,priv,accepted
0,904,200,199,195,198.2,True,True
1,26,193,194,190,192.4,True,True
2,770,193,195,186,191.5,True,True
3,1762,198,193,180,191.1,True,True
4,2202,192,192,182,189.0,True,True
...,...,...,...,...,...,...,...
2395,2044,102,116,107,107.7,False,False
2396,681,104,113,106,107.3,False,False
2397,1798,112,100,108,107.2,False,False
2398,2009,105,111,106,107.1,False,False


*Об'єднання наборів даних*

### Відсортуємо увесь набір даних за стовпцем index, щоб повернути його до того порядку, з якого починали.

In [186]:
df.sort_values(['original_index'], inplace=True, ascending=[True])
df

,original_index,math,eng,ua,score,priv,accepted
1203,0,191,181,102,161.3,False,False
1826,1,161,103,158,142.7,False,False
1018,2,177,194,130,168.0,False,False
709,3,126,115,116,119.7,True,False
1172,4,185,160,134,162.2,False,False
...,...,...,...,...,...,...,...
1522,2395,156,100,196,151.2,False,False
533,2396,161,145,102,138.5,True,False
1717,2397,118,194,134,145.6,False,False
2361,2398,115,107,130,117.1,False,False


*Сортування за стовпцем index*

### Очистимо оперативну пам'ять

In [187]:
%reset -f

*Очистка пам'яті*

## 3.2 Навчання моделі

### Навчання моделі будемо проводити в циклі за допомогою перебору параметрів шарів.

### Модель завжди буде компілюватися з optimizer = 'adam', який працює на основі градієнтного спуску, та функцією втрат loss = 'binary_crossentropy', оскільки ці параметри гарно працюють для бінарної класифікації. 

### Оскільки навчання моделі забирає багато часу та ресурсів, то маємо такі обмеження: максимальна кількість шарів знаходиться у layers_numbers; функціями активації можуть бути лише ті, що наведені в activations; кількість епох знаходиться у epochs_numbers; кількість вхідних нейронів у прихованому шарі знаходиться у neurons_numbers.

### Результати моделей будемо зберігати у список models, що зберігатиме параметри, точності на тренувальних та тестових даних. Запишемо отриманий список у файл.

In [188]:
import numpy as np
import pandas as pd
import copy
layers_numbers = [i for i in range(2, 4)]
activations = ['sigmoid', 'relu']
epochs_numbers = np.array([20, 30, 40]).repeat(3)
neurons_numbers = [50, 100, 200]
def gen_models(layers_numbers, activations,
               epochs_numbers, neurons_numbers):
    models = []
    model = dict(epochs=0, layers=[])
    l = dict(units=0, activation='')
    for layers_num in layers_numbers:
        model['layers'] = [copy.deepcopy(l) for _ in range(layers_num)]
        for epochs in epochs_numbers:
            model['epochs'] = epochs
            for neurons in neurons_numbers:
                for activation in activations:
                    for i, layer in enumerate(model['layers']):
                        layer['units'] = neurons
                        layer['activation'] = activation
                        if i == len(model['layers']) - 1:
                            layer['units'] = 2
                        if i == 0:
                            layer['input_shape'] = (4,)    
                    models.append(format(model))
    return models
    
def write_models(models, output_file):
    with open(output_file, 'w') as file:
        t = '\n'.join(models)
        file.write(t)

*Функція утворення списку можливих комбінацій параметрів*

### Утворимо список.

In [189]:
models = gen_models(layers_numbers, activations,
                    epochs_numbers, neurons_numbers)
write_models(models, 'models.txt')
del models

*Утворення списку можливих комбінацій параметрів*

### Розібравши, як створювати набір даних, запишемо генерування у функцію та утворимо з неї два набори даних тренувальний та тестовий.

In [190]:
import numpy as np
import pandas as pd
def get_dataset(number: int):
    max_applicants = number
    df = pd.DataFrame(
    np.random.randint(100, 200 + 1, (max_applicants, 3)))
    df.columns = ['math', 'eng', 'ua']
    coefs = np.array([0.4, 0.3, 0.3])
    df['score'] = np.apply_along_axis(
        lambda row: np.dot(row, coefs), 1, df.values)
    privilage_quant = 0.3
    privilige = np.random.random_sample((max_applicants, 1))
    privilige = np.apply_along_axis(
        lambda x: x < privilage_quant, 1, privilige)
    df['priv'] = privilige ###
    mins_no_privilige = np.array([140, 120, 120, 160])
    mins_privilige = np.array([120, 120, 120, 144]) 
    def is_score_valid(row: np.ndarray):
        mins = mins_no_privilige
        if row[4]:
            mins = mins_privilige
        return (row[:4] > mins).all()
    df['accepted'] = np.apply_along_axis(
        is_score_valid, 1, df.values) #####
    df.reset_index(inplace=True)
    df.sort_values(['priv', 'score', 'index'], inplace=True,
            ascending=[False, False, True])
    df.reset_index(inplace=True, drop=True)
    df.columns = ['original_index', 'math', 'eng',
                    'ua', 'score', 'priv', 'accepted'] ###
    priv = df.loc[(df.priv == True) & (df.accepted == True)].copy()
    priv.reset_index(inplace=True, drop=True)
    priv = priv[:35] ###
    no_priv = df.loc[~df.original_index.isin(priv.original_index.to_list())].copy()
    no_priv.sort_values(['score', 'original_index'], inplace=True,
            ascending=[False, True])
    no_priv_accepted = no_priv.loc[no_priv.accepted == True]
    no_priv_accepted.reset_index(inplace=True, drop=True)
    no_priv_accepted = no_priv_accepted[:315] ###
    accepted = pd.concat([priv, no_priv_accepted])
    not_accepted = df.loc[~df.original_index.isin(accepted.original_index)]
    not_accepted.accepted = False
    df = pd.concat([accepted, not_accepted])
    df.sort_values(['original_index'], inplace=True, ascending=[True])
    return df.values[:, [1, 2, 3, 4]].astype('float64'), \
        df.values[:, [-1]].astype('float64')

*Функція створення датасету*

### Утворення тренувального та тестового наборів даних.


In [191]:
x_train, y_train = get_dataset(2000)
x_test, y_test = get_dataset(1500)
x_train

/tmp/ipykernel_130136/2251018698.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_accepted.accepted = False
/tmp/ipykernel_130136/2251018698.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_accepted.accepted = False


array([[126.00, 148.00, 155.00, 141.30],
       [172.00, 196.00, 118.00, 163.00],
       [139.00, 138.00, 189.00, 153.70],
       ...,
       [151.00, 162.00, 150.00, 154.00],
       [136.00, 102.00, 189.00, 141.70],
       [155.00, 160.00, 174.00, 162.20]])

*Навчальні та тестові набори даних*


### Підготуємо мітки за допомогою to_categorical. Тобто 0 перетвориться на [1, 0], а 1 - [0, 1].

In [192]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[1.00, 0.00],
       [1.00, 0.00],
       [1.00, 0.00],
       ...,
       [1.00, 0.00],
       [1.00, 0.00],
       [1.00, 0.00]], dtype=float32)

*Підготування міток*

### Нормалізуємо набори даних аргументів. Для початку віднімемо 100 та поділимо на 100.

In [193]:
for arr in [x_train, x_test]:
    arr -= 100
    arr /= 100
x_train

array([[0.26, 0.48, 0.55, 0.41],
       [0.72, 0.96, 0.18, 0.63],
       [0.39, 0.38, 0.89, 0.54],
       ...,
       [0.51, 0.62, 0.50, 0.54],
       [0.36, 0.02, 0.89, 0.42],
       [0.55, 0.60, 0.74, 0.62]])

*Нормалізація даних*

### Запишемо функцію знаходження найкращої моделі.

In [194]:
from ast import literal_eval
from keras import models, layers
best_model = dict(acc=0, model=None)
def find_best_model(input_file, best_model_output_name, best_model):
    with open(input_file, 'r') as file:
        for line in file:
            model = literal_eval(line)
            network = models.Sequential()
            for l in model['layers']:
                network.add(layers.Dense(**l))
            network.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
            network.fit(x_train, y_train, epochs=model['epochs'])
            test_loss, test_acc = network.evaluate(x_test, y_test)
            if best_model['acc'] < test_acc:
                best_model['model'] = model
                best_model['acc'] = test_acc
                with open(f'{best_model_output_name}.txt', 'w') as bb:
                    bb.write(line)
                network.save(f'{best_model_output_name}.h5')
            del network
            del model

*Функція знаходження найкращої моделі*

### Визначимо найкращу модель. Імпортуємо модуль ast для перетворення текстового рядка.

In [195]:
find_best_model('models.txt', 'best_model', best_model)

Epoch 1/20
63/63 [==============================] - 1s 2ms/step - loss: 0.5245 - accuracy: 0.8250
Epoch 2/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4613 - accuracy: 0.8250
Epoch 3/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.8250
Epoch 4/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4397 - accuracy: 0.8250
Epoch 5/20
63/63 [==============================] - 0s 3ms/step - loss: 0.4276 - accuracy: 0.8250
Epoch 6/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4155 - accuracy: 0.8250
Epoch 7/20
63/63 [==============================] - 0s 2ms/step - loss: 0.4024 - accuracy: 0.8250
Epoch 8/20
63/63 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.8250
Epoch 9/20
63/63 [==============================] - 0s 2ms/step - loss: 0.3745 - accuracy: 0.8250
Epoch 10/20
63/63 [==============================] - 0s 2ms/step - loss: 0.3599 - accuracy: 0.8250
Epoch 11/20
63/63 [

KeyboardInterrupt: 

*Визначення найкращої моделі*

### Виведемо найкращу модель.

In [ ]:
best_model

*Найкраща модель*

### Перенесемо прогнози в excel.

In [ ]:
from keras.models import load_model
model = load_model('best_model.h5')
pred = model.predict(x_test)
results = (pred[:, 1] > pred[:, 0]).astype('int32')
results = np.hstack([x_test, results.reshape(results.shape[0], 1)])
results = pd.DataFrame(results)
results.columns = ['math', 'eng', 'ua', 'priv', 'accepted']
results.to_excel('result.xlsx')
results

*Результати моделі*